In [1]:
import geopandas as gpd
import numpy as np
import folium
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
import json
import random
from scipy.interpolate import CubicSpline
from folium.plugins import TimestampedGeoJson
from datetime import datetime, timedelta
from math import sin,cos,atan2,sqrt,radians

In [2]:
types_of_targets_and_start_points = {
    "Shahed": [(46.048989, 38.185922), (53.721409, 33.322867), (51.755399, 36.296972), (45.003586, 35.834130)],
    "X101": [(43.147654, 49.629973), (43.996916, 49.462441), (44.903308, 49.149526)],
    "X555": [(43.147654, 49.629973), (43.996916, 49.462441), (44.903308, 49.149526)],
    "Kalibr": [(44.875215, 32.293704), (45.234660, 30.992986), (44.610035, 35.435840)],
    "X59": [(47.252011, 35.045111), (47.436406, 35.565967), (47.104828, 34.111666), (50.398802, 36.539808)],
    "Iskander": [(47.831649, 39.770591), (52.134637, 32.049890), (51.353375, 34.632992), (45.717054, 33.149679), (47.230451, 35.693289)]}

In [3]:
#назва, дальність виявлення, дальність ураження, кількість ракет, вірогідність виявлення, вірогідність знищення
ppo_characteristics = {"S300": (300, 200, 1000, 0.95, 0.9), "Buk": (140, 120, 1000, 0.9, 0.85), "StormerHMV": (12, 10, 1000, 0.8, 0.85), 
               "Hawk": (120, 100, 1000, 0.85, 0.8), "Spada": (25, 20, 1000, 0.9, 0.88), "SAMPT": (120, 100, 1000, 0.92, 0.9), 
               "NASAMS": (25, 20, 1000, 0.9, 0.9), "Patriot": (180, 160, 1000, 0.95, 0.9), "IRIST": (25, 20, 1000, 0.9, 0.88), 
               "Avenger": (10, 8, 1000, 0.8, 0.85), "Crotale": (12, 10, 1000, 0.8, 0.85)}

targets_speeds = {"Shahed": 180, "X101": 720, "X555":720, "Kalibr":980, "X59":900, "Iskander":2100}

In [4]:
ukraine = gpd.read_file("map_with_war.geojson")

In [5]:
#випадковий вибір кінцевої точки повітряної цілі
def end_points():
    with open('targets.json', 'r') as json_file:
        data = json.load(json_file)
    random_key = random.choice(list(data.keys()))
    targets = data[random_key]
    return targets

#перевірка чи точка знаходиться у межах території України
def is_inside_ukraine(point):
    point = Point(point[1], point[0])
    return ukraine.geometry.contains(point).any()

#генерація наступної точки маршруту повітряної цілі
def generate_new_point(start, angle, distance_km):
    new_point = geodesic(kilometers=distance_km).destination(start, angle)
    return (new_point.latitude, new_point.longitude)

#генерація випадкових координат в межах України для розміщення ппо
def generate_random_coords_within_ukraine():
    ukraine_bounds = ukraine.bounds.iloc[0]
    minx, miny, maxx, maxy = ukraine_bounds
    while True:
        x = np.random.uniform(minx, maxx)
        y = np.random.uniform(miny, maxy)
        point = Point(x, y)
        if ukraine.geometry.contains(point).any():
            return (y, x) 

#розміщення систем ппо, приймає кількість кожного типу систем 
def position_anti_air_systems(**kwargs):
    systems = {}
    for system_name, count in kwargs.items():
        for i in range(1, count+1):
            key = f"{system_name}_{i}"
            systems[key] = generate_random_coords_within_ukraine()
    with open('anti_air_systems.json', 'w') as f:
        json.dump(systems, f, indent=4)
    return systems

#зменшення кількості точок в маршруті цілі, щоб проходитись по меншій кількості значень для пришвидшення симуляції
def reduce_route_detail(air_targets):
    reduced_targets = {}
    for target_name, route in air_targets.items():
        if len(route) % 2 == 0:
            reduced_route = route[::2] 
        else:  
            reduced_route = route[::2] + [route[-1]] 
        if reduced_route[-1] == reduced_route[-2]:  
            reduced_route.pop(-1)  
        reduced_targets[target_name] = reduced_route
    return reduced_targets

#розрахунок дистанції між двома координатами точок
def distance(target_coords,coords):
    R = 6373.0
    dlon = radians(target_coords[1] - coords[1])
    dlat = radians(target_coords[0] - coords[0])
    lat1 = radians(target_coords[0])
    lat2 = radians(coords[0])
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def end_points():
    with open('labeled_targets.json', 'r') as json_file:
        data = json.load(json_file)
    random_key = random.choice(list(data.keys()))
    targets = data[random_key]
    return targets

In [6]:
class AerialTarget:
    def __init__(self, target_type, speed, hight, destruction_prob, max_dist):
        self.target_type = target_type
        self.speed = speed
        self.hight = hight
        self.destruction_prob = destruction_prob
        self.max_dist = max_dist

    #обирається випадкова кінцева і стартова точка від якої дистанція до кінцевої найменша, якщо дистанція більша за максимальну*0.8 то пробуємо обрати кінцеву точку ще 10 разів
    def generate_route(self):
        stop = False
        
        target = random.choice(end_points())
        end_point = (target['latitude'], target['longitude'])
        start_points = types_of_targets_and_start_points[self.target_type]
        distances = [(start_point, distance(start_point, end_point)) for start_point in start_points]
        start_point = min(distances, key=lambda x: x[1])[0]
        counter = 0
        while distance(start_point, end_point) > self.max_dist*0.8:
            target = random.choice(end_points())
            end_point = (target['latitude'], target['longitude'])
            counter+=1
            if counter == 10:
                break
        if counter == 10:
            return None
        route = [start_point]
        current_point = start_point
        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
        step = 20
        total_distance = 0

        #рух цілі поза межами України
        while not is_inside_ukraine(current_point):
            if distance(current_point, end_point) > 3000:
                stop = True
                break
            if self.target_type != "Iskander":
                new_angle = angle_to_b + np.random.uniform(-15, 15)
            else:
                new_angle = angle_to_b
            new_point = generate_new_point(current_point, new_angle, 50)
            distance_moved = distance(current_point, new_point)
            total_distance += distance_moved
            route.append(new_point)
            current_point = new_point
        #рух цілі в межах України, по мірі приближення до кінцевої точки крок зменшується, щоб ціль не пролетіла повз 
        t=0
        while distance(current_point, end_point) > 0.5:
            t+=1
            if t>300:
                return self.generate_route()
            if stop == True:
                break
            else:
                if self.target_type == "Shahed":
                    if distance(current_point, end_point) < 1 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 0.1

                    elif distance(current_point, end_point) < 25 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 1

                    elif distance(current_point, end_point) < 100 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        bias = np.random.choice([-1, 1])
                        new_angle = angle_to_b + bias * np.random.exponential(scale=20)
                        step = 10

                    elif distance(current_point, end_point) < 250 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b + np.random.uniform(-45, 45)
                        step = 20

                    elif distance(current_point, end_point) < 400 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        bias = np.random.choice([-1, 1])
                        new_angle = angle_to_b + bias * (np.random.beta(a=2, b=5) * 90 - 45)
                        step = 20
                    else:
                        if is_inside_ukraine(current_point):
                            new_angle = angle_to_b + np.random.uniform(-60, 60)
                            step = 50
                elif self.target_type == "X101" or self.target_type == "X555" or self.target_type == "Kalibr":
                    if distance(current_point, end_point) < 1 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 0.1
                    elif distance(current_point, end_point) < 25 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 1
                    elif distance(current_point, end_point) < 100 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        bias = np.random.choice([-1, 1])
                        new_angle = angle_to_b + bias * np.random.exponential(scale=10)
                        step = 10
                    elif distance(current_point, end_point) < 250 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b + np.random.uniform(-15, 15)
                        step = 20
                    elif distance(current_point, end_point) < 400 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        bias = np.random.choice([-1, 1])
                        new_angle = angle_to_b + bias * (np.random.beta(a=2, b=5) * 90 - 60)
                        step = 20
                    else:
                        if is_inside_ukraine(current_point):
                            new_angle = angle_to_b + np.random.uniform(-15, 15)
                            step = 50
                elif self.target_type == "X59":
                    if distance(current_point, end_point) < 1 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 0.1
                    elif distance(current_point, end_point) < 25 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b
                        step = 1
                    elif distance(current_point, end_point) < 100 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        bias = np.random.choice([-1, 1])
                        new_angle = angle_to_b + bias * np.random.exponential(scale=10)
                        step = 5
                    elif distance(current_point, end_point) < 250 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b + np.random.uniform(-15, 15)
                        step = 10
                    else:
                        if is_inside_ukraine(current_point):
                            new_angle = angle_to_b + np.random.uniform(-20, 20)
                            step = 10
                elif self.target_type == "Iskander":
                    if distance(current_point, end_point) < 2:
                        break
                    if distance(current_point, end_point) < 10 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b 
                        step = 0.1
                    elif distance(current_point, end_point) < 25 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b 
                        step = 5
                    elif distance(current_point, end_point) < 100 and is_inside_ukraine(current_point):
                        angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                        new_angle = angle_to_b 
                        step = 10
                    else:
                         if is_inside_ukraine(current_point):
                            angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                            new_angle = angle_to_b 
                            step = 50
            
                new_point = generate_new_point(current_point, new_angle, step)
                
                max_attempts = 10
                attempts = 0
                step = 200
                #якщо ціль вилетіла за межі України, то розвертаємо її в напрямку кінцевої точки
                while not is_inside_ukraine(new_point) and attempts < max_attempts:
                    angle_to_b = np.degrees(np.arctan2(end_point[1] - current_point[1], end_point[0] - current_point[0]))
                    new_angle = angle_to_b
                    new_point = generate_new_point(current_point, new_angle, step/2)  
                    attempts += 1

                if attempts == max_attempts:
                    break 

                distance_moved = distance(current_point, new_point)
                total_distance += distance_moved
                route.append(new_point)
                current_point = new_point
        #маршрут згладжується щоб він був плавним
        if total_distance <=self.max_dist and distance(route[-1], end_point) < 10:
            x = [point[0] for point in route]
            y = [point[1] for point in route]
            distances = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
            t = np.concatenate(([0], np.cumsum(distances)))
            cs_x = CubicSpline(t, x)
            cs_y = CubicSpline(t, y)
            t_new = np.linspace(0, t[-1], 75)
            x_new = cs_x(t_new)
            y_new = cs_y(t_new)
            new_coordinates = list(zip(x_new, y_new))
            return new_coordinates

In [7]:
shahid_drone = AerialTarget(target_type="Shahed", speed=180, hight=150, destruction_prob=0.65, max_dist=2000)
x101 = AerialTarget(target_type="X101", speed = 720, hight=110, destruction_prob =0.75, max_dist=5500)
x555 = AerialTarget(target_type="X555", speed = 720, hight=110, destruction_prob =0.85, max_dist=2500)
kalibr = AerialTarget(target_type="Kalibr", speed = 980, hight=150, destruction_prob =0.7, max_dist=300)
x59 = AerialTarget(target_type="X59", speed=900, hight=150, destruction_prob=0.8, max_dist=300)
iskander = AerialTarget(target_type="Iskander", speed=2100, hight=50000, destruction_prob=0.6, max_dist=500)

In [8]:
class PPOSystem:
    def __init__(self, name, coords, detection_range, destruction_range, missile_count, detection_prob, destruction_prob):
        self.name = name
        self.coords = coords
        self.detection_range = detection_range
        self.destruction_range = destruction_range
        self.missile_count = missile_count
        self.base_detection_prob = detection_prob
        self.base_destruction_prob = destruction_prob

    #виявлення цілі, ціль може зникнути з вірогідністю 0.3, також якщо ціль шахід то вірогідність виявлення зменшується, поза межами України вірогідність виявлення 0
    def detect_target(self, target_coords, target_name, air_targets):
        detection_probability = self.base_detection_prob
        if is_inside_ukraine(target_coords):
            if random.random() < 0.3:
                detection_probability = 0
            else:
                if target_name.split('_')[0] == "Shahed":
                    detection_probability = 0.5
                original_route_distance = distance(air_targets[target_name][-1], air_targets[target_name][0])
                remaining_distance = distance(air_targets[target_name][-1], target_coords)
                detection_probability *= (1 - (remaining_distance / original_route_distance))
        else:
            detection_probability = 0
        if self.calculate_distance(self.coords, target_coords) <= self.detection_range and random.random() < detection_probability:
            return True
        return False

    #знищення цілі, балістичні ракети можуть бути знищені тільки 2 типами систем ппо
    def destroy_target(self, target_coords, target_name):
        destruction_probability = self.base_destruction_prob
        if target_name.split('_')[0] == "Iskander" and self.name.split('_')[0] not in ("Patriot", "SAMPT"):
            destruction_probability = 0
        elif target_name.split('_')[0] == "Shahed":
            destruction_probability = 0.5
        if self.missile_count > 0 and self.calculate_distance(self.coords, target_coords) <= self.destruction_range:
            self.missile_count -= 1
            if random.random() < destruction_probability:
                return True
        return False

    def calculate_distance(self, coord1, coord2):
        return distance(coord1, coord2)
    
    def is_in_destruction_range(self, *target_coords):
        return self.calculate_distance(self.coords, target_coords) <= self.destruction_range

In [9]:
#симуляція атаки 
def start_attac(amount_of_shahed=0, amount_of_x101=0, amount_of_x555=0, amount_of_kalibr=0, amount_of_x59=0, amount_of_iskander=0):
    result = {}
    i=0
    while i<amount_of_shahed:
        res = shahid_drone.generate_route()
        if res is not None:
            i+=1
            result["Shahed_"+str(i)] = res
    j=0
    while i<amount_of_x101:
        res = x101.generate_route()
        if res is not None:
            j+=1
            result["X101_"+str(j)] = res
    k=0
    while k<amount_of_x555:
        res = x101.generate_route()
        if res is not None:
            k+=1
            result["X555_"+str(k)] = res
    d=0
    while d<amount_of_kalibr:
        res = kalibr.generate_route()
        if res is not None:
            d+=1
            result["Kalibr_"+str(d)] = res
    m=0
    while m<amount_of_x59:
        res = x59.generate_route()
        if res is not None:
            m+=1
            result["X59_"+str(m)] = res
    f=0
    while f<amount_of_iskander:
        res = iskander.generate_route()
        if res is not None:
            f+=1
            result["Iskander_"+str(f)] = res
    
    with open('result.json', 'w') as json_file:
        json.dump(result, json_file, indent=4)
    return result

In [10]:
#створення карти з маршрутами
def visualisation():
    with open('result.json', 'r') as f:
        routes_data = json.load(f)
    m = folium.Map(location = [49.0139, 31.2858], zoom_start = 5.5)
    for key, route in routes_data.items():
        folium.PolyLine(route, color="blue", weight=2.5, opacity=1).add_to(m)
        end_point = route[-1]
        folium.CircleMarker(
                end_point,
                radius=2, 
                color='blue', 
                fill=True,
                fill_color='blue',
                fill_opacity=0.7,
                tooltip=key 
            ).add_to(m)
    m.save('static_map.html')

In [11]:
#створення анімованої карти з маршрутами
def animation():
    with open('result.json', 'r') as f:
        routes_data = json.load(f)
    m = folium.Map(location = [49.0139, 31.2858], zoom_start = 5.5)
    all_features = []
    start_time = datetime.now()
    for route_name, route_coords in routes_data.items():
        for i, coord in enumerate(route_coords):
            routes_data[route_name][i] = (coord[1], coord[0])
    for route_name, route_coords in routes_data.items():
        features = [
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [route_coords[i], route_coords[i + 1]],
                },
                "properties": {
                    "times": [
                        int(start_time.timestamp() * 1000),
                        int((start_time + timedelta(seconds=(i) * 1)).timestamp() * 1000)
                    ],
                    "style": {
                        "color": "blue",
                        "weight": 3
                    }
                }
            }
            for i in range(len(route_coords) - 1)
        ]
        all_features.extend(features)

    TimestampedGeoJson(
        {"type": "FeatureCollection", "features": all_features},
        period="PT3S",
        add_last_point=False,
        auto_play=False,
        loop=False,
        max_speed=25,
        loop_button=True,
        date_options='YYYY/MM/DD HH:mm:ss',
        time_slider_drag_update=True,
    ).add_to(m)
    m.save('animated_map.html')

In [12]:
colors = {"S300": "blue", "Buk": "blue", "StormerHMV": "green", 
               "Hawk": "blue", "Spada": "green", "SAMPT": "red", 
               "NASAMS": "green", "Patriot": "red", "IRIST": "green", 
               "Avenger": "green", "Crotale": "green"}

#розміщення систем ппо на карті
def ppo_visualisation(strategy):
    with open(f'{strategy}/anti_air_systems.json', 'r') as f:
        systems = json.load(f)
    m = folium.Map(location = [49.0139, 31.2858], zoom_start = 5.5)
    for system_name, coords in systems.items():
        color = colors[system_name.split("_")[0]]
        folium.Marker(
            location=coords,
            tooltip=system_name,  
            icon=folium.Icon(color=color, icon="star")
        ).add_to(m)
        folium.Circle(
                    coords,
                    radius=ppo_characteristics[system_name.split("_")[0]][1]*1000, 
                    color='green', 
                    fill=True,
                    fill_color='green',
                    fill_opacity=0.25,
                    tooltip=system_name
                ).add_to(m)
    m.save("systems_map.html")

In [13]:
#розрахунок зон виявлення і ураження усіх систем ппо
def calculate_ppo_ranges(systems):
    ppo_ranges = {}
    for system in systems:
        detection_circle = Point(system.coords).buffer(system.detection_range)
        destruction_circle = Point(system.coords).buffer(system.destruction_range)
        ppo_ranges[system.name] = {
            'detection': detection_circle,
            'destruction': destruction_circle
        }
    return ppo_ranges

In [14]:
#симуляція атаки і збиття
def simulate_defence(idx, strategy):
    with open('result.json', 'r') as f:
        air_targets = json.load(f)
    with open(f'{strategy}/anti_air_systems.json', 'r') as f:
        ppo_systems = json.load(f)
    with open('labeled_targets.json', 'r') as f:
        labeled = json.load(f)
        
    air_targets = reduce_route_detail(air_targets)

    systems = [PPOSystem(system_name, coords, *ppo_characteristics[system_name.split('_')[0]])
               for system_name, coords in ppo_systems.items()]

    ppo_ranges = calculate_ppo_ranges(systems)
    
    destroyed_targets = set() 
    destroyed_targets_info = {} 
    systems_kills = {}
    max_route_length = max([len(route) for route in air_targets.values()])

    for i in range(max_route_length):
        for target_name, route in air_targets.items():
            if target_name in destroyed_targets:
                continue
            if i < len(route):
                target_coords = route[i]
                for system in systems:
                    if not is_inside_ukraine(target_coords):
                        continue

                    if Point(target_coords).within(ppo_ranges[system.name]['detection']):
                        if system.detect_target(target_coords, target_name, air_targets) and Point(target_coords).within(ppo_ranges[system.name]['destruction']):
                            if system.destroy_target(target_coords, target_name):
                                systems_kills[system.name] = target_name
                                destroyed_targets.add(target_name)
                                destroyed_targets_info[target_name] = i  
                                break  
    dt =  list(destroyed_targets)
    st =  [name for name in air_targets.keys() if name not in destroyed_targets]
    percent = len(dt)/(len(dt)+len(st))
#     print("Destroyed targets:", len(dt), dt)
#     print("Survived targets:", len(st), st)
#     print("Percent of destroyed targets:", percent)
    visualize_routes(air_targets, destroyed_targets_info, ppo_systems, idx, strategy) 
    critical_damage = 0
    for i in st:
        lat = round(air_targets[i][-1][0], 2)
        long = round(air_targets[i][-1][1], 2)
        for j in labeled:
            for k in labeled[j]:
                if round(k["latitude"], 2) == lat and round(k["longitude"], 2) == long:
                    critical_damage += k["critical_rate"]
#     print("Critical damage", critical_damage)
    return percent, critical_damage

In [15]:
#візуалізаці маршртутів збитих і не збитих цілей, не збиті червоним
def visualize_routes(air_targets, destroyed_targets, ppo_systems, idx, strategy):
    m = folium.Map(location=[48.3794, 31.1656], zoom_start=6)  
    for system_name, coords in ppo_systems.items():
        color = colors[system_name.split("_")[0]]
        folium.Marker(
            location=coords,
            tooltip=system_name,  
            icon=folium.Icon(color=color, icon="star")
        ).add_to(m)
        folium.Circle(
                    coords,
                    radius=ppo_characteristics[system_name.split("_")[0]][1]*1000, 
                    color='green', 
                    fill=True,
                    fill_color='green',
                    fill_opacity=0.25,
                    tooltip=system_name
                ).add_to(m)
    for target_name, route in air_targets.items():
        if target_name in destroyed_targets:
            folium.PolyLine(route[:destroyed_targets[target_name]+1], color="blue", weight=2.5, opacity=1).add_to(m)
            end_point = route[:destroyed_targets[target_name]+1][-1]
            folium.CircleMarker(
                    end_point,
                    radius=2, 
                    color='blue', 
                    fill=True,
                    fill_color='blue',
                    fill_opacity=0.7,
                    tooltip=target_name
                ).add_to(m)
        else:
            folium.PolyLine(route, color="red", weight=2.5, opacity=1).add_to(m)
            end_point = route[-1]
            folium.CircleMarker(
                    end_point,
                    radius=2, 
                    color='red', 
                    fill=True,
                    fill_color='red',
                    fill_opacity=0.7,
                    tooltip=target_name
                ).add_to(m)

    m.save(f'{strategy}/routes_map_{idx}.html')